In [ ]:
import csv
import cv2
import pandas as pd
from ultralytics import YOLO
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
import gdown
import zipfile
import os
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg
from PIL import Image
import numpy as np
import random
import shutil
import yaml
from PIL import Image
import os
import random
import matplotlib.pyplot as plt
from ultralytics import YOLO

warnings.simplefilter(action='ignore', category=FutureWarning)

# Define constants
HOME = os.getcwd()
DATASETS_DIR = os.path.join(HOME, 'datasets')

# Create datasets directory if it doesn't exist
if not os.path.exists(DATASETS_DIR):
    os.makedirs(DATASETS_DIR)

# Change directory to datasets directory
os.chdir(DATASETS_DIR)

# Print current working directory
print(os.getcwd())

In [ ]:
from google.colab import auth
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

# Autentikasi pengguna
auth.authenticate_user()

# Otorisasi dan autentikasi untuk Google Drive
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()

# Membuat koneksi ke Google Drive
drive = GoogleDrive(gauth)

# ID file dari tautan
file_id = "1QDCT7w0TaG7hTu33Kcm-wBxGrsV9UEz9"
file_name = "ships-aerial-images"  # Nama file untuk menyimpan unduhan

try:
    print(f"Mengunduh file dengan ID {file_id}...")
    # Mengunduh file berdasarkan ID
    downloaded = drive.CreateFile({'id': file_id})
    downloaded.GetContentFile(file_name)  # Menyimpan file dengan nama yang ditentukan
    print(f"File berhasil diunduh sebagai {file_name}!")
except Exception as e:
    print(f"Terjadi error saat mengunduh file: {e}")


In [ ]:
!unzip /content/datasets/ships-aerial-images -d /content/data

In [ ]:

import os

def count_items_in_directory(directory):
    try:
        return sum(len(files) for _, _, files in os.walk(directory))
    except FileNotFoundError:
        return 0

def count_items(base_path):
    categories = ['train', 'valid', 'test']
    classes = ["images", 'labels']
    total_images = 0

    for category in categories:
        print(f"--- {category.capitalize()} Data ---")
        category_path = os.path.join(base_path, category)
        for class_name in classes:
            class_path = os.path.join(category_path, class_name)
            num_items = count_items_in_directory(class_path)
            total_images += num_items
            if num_items > 0:
                print(f"{class_name.capitalize()}: {num_items} images")
            else:
                print(f"{class_name.capitalize()}: Directory does not exist or is empty")

    print(f"\nTotal Images: {total_images} images")

# Define base path
base_path = '/content/data'
count_items(base_path)


In [ ]:
import os
from PIL import Image

def check_images_in_directory(directory):
    # Daftar ekstensi gambar yang valid
    valid_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']

    for root, _, files in os.walk(directory):
        for file in files:
            # Cek apakah file memiliki ekstensi gambar yang valid
            if any(file.lower().endswith(ext) for ext in valid_extensions):
                try:
                    img_path = os.path.join(root, file)
                    img = Image.open(img_path)
                    img.verify()  # Verifikasi apakah file adalah gambar yang valid
                except (IOError, Image.UnidentifiedImageError) as e:
                    print(f"File tidak valid ditemukan: {img_path}, Error: {e}")
            else:
                # File teks atau file yang bukan gambar akan diabaikan
                continue

# Memeriksa dataset
check_images_in_directory("/content/data/train")
check_images_in_directory("/content/data/valid")


In [ ]:
def load_image_and_labels(image_path, label_path):
    """Memuat gambar dan label yang sesuai dari path yang diberikan."""
    # Membaca gambar
    image = cv2.imread(image_path)

    # Membaca label
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            labels = f.read().strip().split("\n")
    else:
        labels = []

    return image, labels

def draw_bounding_boxes(image, labels):
    """Menggambar kotak pembatas pada gambar berdasarkan label."""
    for label in labels:
        # Pastikan label memiliki format yang benar
        parts = label.split()
        if len(parts) != 5:
            continue

        # Mengambil informasi bounding box
        class_id, x_center, y_center, width, height = map(float, parts)

        # Menghitung koordinat bounding box
        x_min = int((x_center - width / 2) * image.shape[1])
        y_min = int((y_center - height / 2) * image.shape[0])
        x_max = int((x_center + width / 2) * image.shape[1])
        y_max = int((y_center + height / 2) * image.shape[0])

        # Menggambar kotak pembatas
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 3)

    return image

def plot_images(images, ncols=4):
    """Menampilkan gambar-gambar dalam format grid."""
    nrows = len(images) // ncols + (1 if len(images) % ncols != 0 else 0)
    fig, axs = plt.subplots(nrows, ncols, figsize=(16, 16))
    axs = axs.ravel()  # Meratakan array 2D dari axes untuk memudahkan iterasi

    for i, (image, ax) in enumerate(zip(images, axs)):
        ax.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        ax.axis('off')

    # Menyembunyikan subplot yang tidak terpakai
    for i in range(len(images), len(axs)):
        axs[i].axis('off')

    plt.tight_layout()
    plt.show()

# Menentukan path ke direktori
train_images = "/content/data/train/images"
train_labels = "/content/data/train/labels"

# Mendapatkan daftar semua file gambar di direktori train
image_files = os.listdir(train_images)

# Memilih 16 file gambar secara acak
random_images = random.sample(image_files, 16)

# Menyiapkan gambar untuk ditampilkan
images_to_plot = []

# Loop untuk memproses gambar acak dan menggambar deteksi objek
for image_file in random_images:
    # Menentukan path file gambar dan label
    image_path = os.path.join(train_images, image_file)
    label_file = os.path.splitext(image_file)[0] + ".txt"
    label_path = os.path.join(train_labels, label_file)

    # Memuat gambar dan label
    image, labels = load_image_and_labels(image_path, label_path)

    # Menggambar kotak pembatas pada gambar
    image_with_boxes = draw_bounding_boxes(image, labels)

    # Menambahkan gambar yang telah diberi kotak pembatas ke daftar gambar
    images_to_plot.append(image_with_boxes)

# Menampilkan gambar-gambar dengan kotak pembatas
plot_images(images_to_plot)


In [ ]:
# Load an image using OpenCV
image = cv2.imread("/content/data/test/images/02e39612d_jpg.rf.cc5483bb711f080d12b644ff62cf977a.jpg")

# Get the size of the image
height, width, channels = image.shape
print(f"The image has dimensions {width}x{height} and {channels} channels.")

In [ ]:

data = {
    'train': '/content/data/train/images',
    'val': '/content/data/valid/images',
    'test': '/content/data/test/images',
    'nc': 1,
    'names': ['ship']
}

yaml_file_path = '/content/data/data.yaml'

with open(yaml_file_path, 'w') as yaml_file:
    yaml.dump(data, yaml_file, default_flow_style=False, sort_keys=False)

print(f"File YAML berhasil disimpan di {yaml_file_path}")


# **PERSIAPAN ANALISIS**

In [ ]:
train_images_dir = '/content/data/train/images'
train_labels_dir = '/content/data/train/labels'

def is_valid_coordinates(coords, image_width, image_height):
    """Periksa apakah koordinat bounding box berada dalam rentang [0, 1]"""
    class_id, x_center, y_center, width, height = coords
    # Cek jika koordinat bounding box di luar rentang yang valid
    return (0 <= x_center <= 1) and (0 <= y_center <= 1) and (0 <= width <= 1) and (0 <= height <= 1)

def remove_invalid_images_and_labels():
    files_to_remove = []
    for image_file in os.listdir(train_images_dir):
        image_path = os.path.join(train_images_dir, image_file)
        label_file = os.path.splitext(image_file)[0] + '.txt'
        label_path = os.path.join(train_labels_dir, label_file)

        if os.path.exists(label_path):
            with open(label_path, 'r') as label_f:
                lines = label_f.readlines()
            for line in lines:
                coords = list(map(float, line.strip().split()))
                if len(coords) == 5:
                    if not is_valid_coordinates(coords, image_width=1, image_height=1):
                        print(f"Invalid coordinates in {image_file}: {coords}")
                        files_to_remove.append(image_path)
                        files_to_remove.append(label_path)
                        break

    for file in files_to_remove:
        os.remove(file)
        print(f"Removed: {file}")

remove_invalid_images_and_labels()


In [ ]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()


In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

os.environ['WANDB_MODE'] = 'disabled'

In [ ]:
model = YOLO('yolov8x.pt')
model.train(data = '/content/data/data.yaml',
            epochs = 20,
            imgsz = 256,
            seed = 42,
            batch = 8,
            workers = 4)

# !yolo task=detect mode=train model=yolov9m.pt data="/content/data/data.yaml" epochs=20 imgsz=256 batch=32 device=0 workers=8

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Membaca file CSV
df = pd.read_csv('/content/datasets/datasets/runs/detect/train/results.csv')
df.columns = df.columns.str.strip()  # Menghapus spasi tambahan di nama kolom

# Membuat subplot dengan seaborn
fig, axs = plt.subplots(nrows=5, ncols=2, figsize=(15, 15))

# Plot untuk data train
sns.lineplot(x='epoch', y='train/box_loss', data=df, ax=axs[0,0])
sns.lineplot(x='epoch', y='train/cls_loss', data=df, ax=axs[0,1])
sns.lineplot(x='epoch', y='train/dfl_loss', data=df, ax=axs[1,0])
sns.lineplot(x='epoch', y='metrics/precision(B)', data=df, ax=axs[1,1])
sns.lineplot(x='epoch', y='metrics/recall(B)', data=df, ax=axs[2,0])
sns.lineplot(x='epoch', y='metrics/mAP50(B)', data=df, ax=axs[2,1])
sns.lineplot(x='epoch', y='metrics/mAP50-95(B)', data=df, ax=axs[3,0])
sns.lineplot(x='epoch', y='val/box_loss', data=df, ax=axs[3,1])
sns.lineplot(x='epoch', y='val/cls_loss', data=df, ax=axs[4,0])
sns.lineplot(x='epoch', y='val/dfl_loss', data=df, ax=axs[4,1])

# Menetapkan judul dan label sumbu untuk setiap subplot
axs[0,0].set(title='Train Box Loss')
axs[0,1].set(title='Train Class Loss')
axs[1,0].set(title='Train DFL Loss')
axs[1,1].set(title='Metrics Precision (B)')
axs[2,0].set(title='Metrics Recall (B)')
axs[2,1].set(title='Metrics mAP50 (B)')
axs[3,0].set(title='Metrics mAP50-95 (B)')
axs[3,1].set(title='Validation Box Loss')
axs[4,0].set(title='Validation Class Loss')
axs[4,1].set(title='Validation DFL Loss')

# Menambahkan suptitle untuk seluruh grafik
plt.suptitle('Training Metrics and Loss', fontsize=24)

# Menyesuaikan margin atas untuk memberi ruang pada suptitle
plt.subplots_adjust(top=0.8)

# Menyesuaikan jarak antar subplot
plt.tight_layout()

# Menampilkan plot
plt.show()


#Evaluation

In [ ]:
model = YOLO('/content/datasets/datasets/runs/detect/train/weights/best.pt')
metrics = model.val(split='test')  # Evaluasi model pada dataset test

# Akses metrik menggunakan results_dict
results = metrics.results_dict

# Print the evaluation metrics with correct keys
print("Evaluation Metrics:")
print(f"Precision: {results['metrics/precision(B)']}")
print(f"Recall: {results['metrics/recall(B)']}")
print(f"mAP50: {results['metrics/mAP50(B)']}")
print(f"mAP50-95: {results['metrics/mAP50-95(B)']}")


In [ ]:
ax = sns.barplot(x=['mAP50-95', 'mAP50', 'mAP75'],
                 y=[metrics.box.map, metrics.box.map50, metrics.box.map75],
                 palette='Blues')

ax.set_title('YOLO Evaluation Metrics')
ax.set_xlabel('Metric')
ax.set_ylabel('Value')

fig = plt.gcf()
fig.set_size_inches(8, 6)

for p in ax.patches:
    ax.annotate('{:.3f}'.format(p.get_height()), (p.get_x() + p.get_width() / 2, p.get_height()), ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Reading the confusion matrix image file
img = mpimg.imread('/content/datasets/datasets/runs/detect/train/confusion_matrix_normalized.png')

# Plotting the confusion matrix image
fig, ax = plt.subplots(figsize = (12, 10))

# Display the image
ax.imshow(img)

# Turn off axis
ax.axis('off')

# Display the plot
plt.show()

# **PREDIC**

In [ ]:
model = YOLO('/content/datasets/datasets/runs/detect/train/weights/best.pt')

# Fungsi deteksi gambar
def ship_detect(image_path):
    img = Image.open(image_path)
    results = model.predict(source=image_path)
    detect_img = results[0].plot()  # plot() akan menggambar kotak deteksi pada gambar
    return detect_img

custom_image_dir = '/content/data/test/images'

# Pilih 16 gambar secara acak
image_files = os.listdir(custom_image_dir)
selected_images = random.sample(image_files, 16)

# Membuat subplot 4x4
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(15, 15))

# Menampilkan hasil deteksi untuk setiap gambar
for i, img_file in enumerate(selected_images):
    row_idx = i // 4
    col_idx = i % 4
    img_path = os.path.join(custom_image_dir, img_file)

    # Deteksi objek pada gambar
    detect_img = ship_detect(img_path)

    # Menampilkan gambar deteksi pada subplot
    axes[row_idx, col_idx].imshow(detect_img)
    axes[row_idx, col_idx].axis('off')

# Menyesuaikan jarak antar gambar
plt.subplots_adjust(wspace=0.05, hspace=0.05)
plt.tight_layout()
plt.show()


In [ ]:
!yolo task=detect mode=predict max_det=1 model=/content/datasets/datasets/runs/detect/train/weights/best.pt conf=0.10 source=/content/1.jpg